In [1]:
# Import
#TODO: se nel df iniziale ci sono le colonne target, assicurarsi di non perderle dopo il bucketing. Creato il filtro per farlo ma serve il fix sui filters vuoti
from pyspark.sql import SparkSession
from ts_train.tr2ts.time_bucketing import TimeBucketing
from ts_train.tr2ts.aggregating import Aggregating, Aggregation, Filter, AndGroup
from ts_train.tr2ts.filling import Filling
import numpy as np 
from pyspark.sql import functions as F
import pandas as pd

# Codice per visualizzazione su notebook
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

path_to_data = "../../../dataset_offline/tr2ft/demo_v1/"
DATA_COLUMN_NAME = "DATA_TRANSAZIONE"

# Create a SparkSession
spark = SparkSession.builder \
    .appName("feature_generation") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/07 21:12:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/07 21:12:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


23/09/07 21:13:05 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Load Positive transactions

In [2]:
# load positive dataset 
# 100k user for 39.682.111 40 million transactions
positive_df_path = path_to_data + "positive_target/100k_user_transactions.parquet"
positive_tr_df = spark.read.parquet(positive_df_path)
positive_tr_df = positive_tr_df.withColumn("TARGET", F.lit(1))

print(positive_tr_df.count())
print(positive_tr_df.select("ID_CLIENTE_BIC").distinct().count())


39682110


99746


# Load Negative transactions

In [3]:
# 100k user for 960.404 1 million transactions
negative_df_path = path_to_data + "negative_target/sample_transactions.parquet"
negative_tr_df = spark.read.parquet(negative_df_path)
negative_tr_df = negative_tr_df.withColumn("TARGET", F.lit(0))
print(negative_tr_df.count())
print(negative_tr_df.select("ID_CLIENTE_BIC").distinct().count())


960404


23/09/07 11:09:24 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


1000


# Create full transaction dataset

In [5]:
# Join positive and negative dataset
tr_df = positive_tr_df.union(negative_tr_df)
# targets
targets_df = tr_df.select("ID_CLIENTE_BIC","TARGET").distinct()
tr_df = tr_df.drop("TARGET")


tr_df.write.parquet(path_to_data + "transaction_dataset_df.parquet")
targets_df.write.parquet(path_to_data + "targets_df.parquet")

23/09/07 11:09:49 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/09/07 11:10:02 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/09/07 11:10:03 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/09/07 11:10:03 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
23/09/07 11:10:03 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


# Load dataset

In [13]:
tr_df = spark.read.parquet(path_to_data + "transaction_dataset_df.parquet")
targets_df = spark.read.parquet(path_to_data + "targets_df.parquet")

In [28]:
tr_df.count()


40642514

# Filter dataset
2000 utenti per un totale di 1 milione di transazioni

In [38]:
TOTAL_ID_TO_MAINTAIN = 1000
positive_id_to_mantain = targets_df.filter(targets_df.TARGET == 1).distinct().limit(TOTAL_ID_TO_MAINTAIN)
negative_id_to_mantain = targets_df.filter(targets_df.TARGET == 0).distinct().limit(TOTAL_ID_TO_MAINTAIN)

filtered_ids = positive_id_to_mantain.union(negative_id_to_mantain)

# filtet tr_df with positive_id_to_mantain
filterd_df = tr_df.join(filtered_ids, on="ID_CLIENTE_BIC", how="inner")
print("filtered: ", filterd_df.count())


original:  40642514
filtered:  1369803


# Transactions to timeseries

In [39]:
# TimeBucketing
time_bucket_step = TimeBucketing(
    time_column_name=DATA_COLUMN_NAME,
    time_bucket_size=1,
    time_bucket_granularity="day", 
)
time_bucket_df = time_bucket_step(filterd_df, spark)
#print(time_bucket_df.count())
time_bucket_df.show(truncate=False)


+--------------+---------------+--------------+----------------+-------------------+-------+-----+--------+-----------+---------------+------+------+-------------------+-----------------------+-------------+-----+------+-----------+------+-------------------+-------------------+
|ID_CLIENTE_BIC|ARCA_TIPO_CARTA|DATA_CONTABILE|DATA_TRANSAZIONE|ORA_TRANSAZIONE    |IMPORTO|SEGNO|IS_CARTA|TIPO_CANALE|TIPO_CANALE_AGG|IS_BON|IS_SDD|CATEGORY_LIV0      |CATEGORY_LIV1          |CATEGORY_LIV2|IS_CC|IS_LIB|MERCHANT   |TARGET|bucket_start       |bucket_end         |
+--------------+---------------+--------------+----------------+-------------------+-------+-----+--------+-----------+---------------+------+------+-------------------+-----------------------+-------------+-----+------+-----------+------+-------------------+-------------------+
|86772451      |EVOLUTION      |2021-07-08    |2020-12-17      |2023-08-28 14:36:26|3.5    |+    |true    |ALTRO      |ALTRO          |false |false |shopping   

In [40]:
# Aggregating
aggregation_step = Aggregating(
    identifier_cols_name=["ID_CLIENTE_BIC"],
    time_bucket_cols_name=["bucket_start", "bucket_end"],
    aggregations=[
        Aggregation(
            numerical_col_name="IMPORTO",
            agg_function="sum",
            filters=None,
            new_col_name="somma_importi",
        ),
        
    ],
)

all_aggregations_df = aggregation_step(time_bucket_df, spark)
print(all_aggregations_df.show())

+--------------+-------------------+-------------------+-------------+
|ID_CLIENTE_BIC|       bucket_start|         bucket_end|somma_importi|
+--------------+-------------------+-------------------+-------------+
|      35931425|2021-07-02 00:00:00|2021-07-02 23:59:59|        258.0|
|      35931425|2021-07-03 00:00:00|2021-07-03 23:59:59|        108.0|
|      35931425|2021-07-04 00:00:00|2021-07-04 23:59:59|         55.0|
|      35931425|2021-07-05 00:00:00|2021-07-05 23:59:59|        230.0|
|      35931425|2021-07-06 00:00:00|2021-07-06 23:59:59|          0.0|
|      35931425|2021-07-10 00:00:00|2021-07-10 23:59:59|        610.0|
|      35931425|2021-07-12 00:00:00|2021-07-12 23:59:59|       2458.0|
|      35931425|2021-07-16 00:00:00|2021-07-16 23:59:59|         46.0|
|      35931425|2021-07-19 00:00:00|2021-07-19 23:59:59|          0.0|
|      35931425|2021-07-20 00:00:00|2021-07-20 23:59:59|         14.0|
|      35931425|2021-07-23 00:00:00|2021-07-23 23:59:59|         32.0|
|     

In [41]:
# Filling
filling_step = Filling(
    identifier_cols_name=["ID_CLIENTE_BIC"],
    time_bucket_step=time_bucket_step
)

filled_df = filling_step(df=all_aggregations_df, spark=spark)
#print(filled_df.count())

In [42]:
# Write Time series df
# dump to parquet
filled_df.write.parquet(path_to_data + "dataset_timeseries.parquet")

23/09/07 12:49:53 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [43]:
filled_df.count()

1404671

In [4]:
# Load TimeSeries dataset
from ts_train.ts2ft.feature_generating import FeatureGenerating
time_series_df = spark.read.parquet(path_to_data + "filtered_dataset_timeseries.parquet")



In [ ]:

#lowered_time_series_df.show(truncate=False)
# Create FeatureGenerating step
feature_gen_step = FeatureGenerating(
    identifier_col_name="ID_CLIENTE_BIC",
    time_col_name = "bucket_start",
    features_calculators= [
        'longest_strike_below_mean',
        'benford_correlation',
        'cid_ce',
        'sum_values',
        'standard_deviation',
        'number_peaks',
        'longest_strike_below_mean',
        'abs_energy',
        'absolute_sum_of_changes',
        'agg_autocorrelation',
        'agg_linear_trend',
        'augmented_dickey_fuller',
        'kurtosis',
        'large_standard_deviation',
        'mean_second_derivative_central',
        'query_similarity_count',
        'cid_ce',
        'count_above'
    ]

)

features_generated_df = feature_gen_step(time_series_df)
features_generated_df.write.parquet(path_to_data + "generated_features.parquet")


In [34]:
from ts_train.ts2ft.feature_generating import FeatureGenerating
from ts_train.ts2ft.feature_pruning import FeaturePruning

In [35]:
generated_features_df = spark.read.parquet(path_to_data + "generated_features_2.parquet")

In [36]:
targets_df = spark.read.parquet(path_to_data + "targets_df.parquet")

In [39]:
generated_features_df = generated_features_df.join(targets_df, on="ID_CLIENTE_BIC", how="inner")
generated_features_df = generated_features_df.toPandas()

In [40]:
# drop target columns from generated_features_df
pandas_feats_df = generated_features_df.drop(["TARGET"], axis=1)

In [41]:
# create a pandas.Series
targets = pd.Series(generated_features_df["TARGET"].values)

In [42]:
targets

0       0
1       0
2       0
3       0
4       1
       ..
2001    0
2002    1
2003    1
2004    0
2005    0
Length: 2006, dtype: int32

In [43]:
feature_pruning_step = FeaturePruning(
  identifier_col_name="ID_CLIENTE_BIC"
)

pruned_df, relevance_table = feature_pruning_step(pandas_feats_df, targets)

/Users/lucasorrentino/miniconda3/envs/ts_train/lib/python3.11/site-packages/ts_train/ts2ft/feature_pruning.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(0, self.identifier_col_name, identifier_col)


In [49]:
# dump relevance_table to file
relevance_table.to_csv(path_to_data + "relevance_table.csv", index=False)
pruned_df.to_csv(path_to_data + "pruned_df.csv", index=False)

In [47]:
len(pruned_df.columns)

129

In [48]:
len(generated_features_df.columns)

198